# Bunny ICP

In [54]:
# Imports
import numpy as np
from pydrake.all import PointCloud, Rgba, RigidTransform, RotationMatrix, StartMeshcat
from scipy.spatial import KDTree

from manipulation import FindResource
from manipulation.exercises.grader import Grader
from manipulation.exercises.pose.test_icp import TestICP

In [61]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


## Problem Description
In the lecture, we learned about the Iterative Closest Point (ICP) algorithm. In this exercise, you will implement the ICP algorithm to solve the standard Stanford Bunny problem!

**These are the main steps of the exercise:**
1. Implement the ```least_squares_transform``` function to optimize transformation given correspondence
2. Implement the ```icp``` algorithm using the functions implemented above.

Let's first visualize the point clouds of Stanford bunny in meshcat!

In [82]:
# Visualize Stanford Bunny
xyzs = np.load(FindResource("models/bunny/bunny.npy"))
cloud = PointCloud(xyzs.shape[1])
cloud.mutable_xyzs()[:] = xyzs

# Pose for the blue bunny
X_blue = RigidTransform(RotationMatrix.MakeXRotation(np.pi / 6), [-0.1, 0.1, 0.1])

pointcloud_model = xyzs
pointcloud_scene = X_blue.multiply(xyzs)

meshcat.Delete()
meshcat.SetProperty("/Background", "visible", False)
meshcat.SetProperty("/Cameras/default/rotated/<object>", "zoom", 10.5)
meshcat.SetObject("red_bunny", cloud, point_size=0.01, rgba=Rgba(1.0, 0, 0))
meshcat.SetTransform("red_bunny", RigidTransform())
meshcat.SetObject("blue_bunny", cloud, point_size=0.01, rgba=Rgba(0, 0, 1.0))
meshcat.SetTransform("blue_bunny", X_blue)

## Point cloud registration with known correspondences

In this section, you will follow the [derivation](http://manipulation.csail.mit.edu/pose.html#registration) to solve the optimization problem below.

$$\begin{aligned} \min_{p\in\mathbb{R}^3,R\in\mathbb{R}^{3\times3}} \quad & \sum_{i=1}^{N_s} \| p + R \hspace{.1cm} {^Op^{m_{c_i}}} - p^{s_i}\|^2, \\ s.t. \quad & RR^T = I, \quad \det(R)=1\end{aligned}$$

The goal is to find the transform that registers the point clouds of the model and the scene, assuming the correspondence is known.  You may refer to the implementation from [deepnote](https://deepnote.com/workspace/Manipulation-ac8201a1-470a-4c77-afd0-2cc45bc229ff/project/4-Geometric-Pose-Estimation-cc6340f5-374e-449a-a195-839a3cedec4a/%2Ficp.ipynb) and the explanation from [textbook](http://manipulation.csail.mit.edu/pose.html#icp).

In the cell below, implement the ```least_squares_transform``` nethod.

In [92]:
def least_squares_transform(scene, model) -> RigidTransform:
    """
    Calculates the least-squares best-fit transform that maps corresponding
    points scene to model.
    Args:
      scene: 3xN numpy array of corresponding points
      model: 3xM numpy array of corresponding points
    Returns:
      X_BA: A RigidTransform object that maps point_cloud_A on to point_cloud_B
            such that
                        X_BA.multiply(model) ~= scene,
    """
    # Calculate the central points
    p_s = scene
    p_Omc = model
    p_Ombar = np.vstack(model.mean(axis=1))
    p_sbar = np.vstack(scene.mean(axis=1))
    
    # Calculate the "error" terms, and form the data matrix
    merr = p_Omc - p_Ombar
    serr = p_s - p_sbar
    W = np.matmul(serr, merr.T)
    
    # Compute R
    U, Sigma, Vt = np.linalg.svd(W)
    R = np.matmul(U, Vt)
    if np.linalg.det(R) < 0:
        print("fixing improper rotation")
        Vt[-1, :] *= -1
        R = np.matmul(U, Vt)
        
    # Compute p
    p = p_sbar - np.matmul(R, p_Ombar)

    X_BA = RigidTransform(RotationMatrix(R), p)
    
    ##################
    # your code here
    ##################
    return X_BA

## Point correspondence from closest point
The ```least_squares_transform``` function assumes that the point correspondence is known. Unfortunately, this is often not the case, so we will have to estimate the point correspondence as well. A common heuristics for estimating point correspondence is the closest point/nearest neighbor.

We have implemented the closest neighbors using [scipy's implementation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.html) of [k-d trees](https://en.wikipedia.org/wiki/K-d_tree).

In [93]:
def nearest_neighbors(scene, model):
    """
    Find the nearest (Euclidean) neighbor in model for each
    point in scene
    Args:
        scene: 3xN numpy array of points
        model: 3xM numpy array of points
    Returns:
        distances: (N, ) numpy array of Euclidean distances from each point in
            scene to its nearest neighbor in model.
        indices: (N, ) numpy array of the indices in model of each
            scene point's nearest neighbor - these are the c_i's
    """    
    kdtree = KDTree(model.T)

    distances, indices = kdtree.query(scene.T, k=1)

    return distances.flatten(), indices.flatten()

## Iterative Closest Point (ICP)
Now you should be able to register two point clouds iteratively by first finding/updating the estimate of point correspondence with ```nearest_neighbors``` and then computing the transform using ```least_squares_transform```. You may refer to the explanation from [textbook](http://manipulation.csail.mit.edu/pose.html#icp).

**In the cell below, complete the implementation of ICP algorithm using the  ```nearest_neighbors``` and ```least_squares_transform``` methods from above.**

In [ ]:
def icp(scene, model, max_iterations=20, tolerance=1e-3):
    """
    Perform ICP to return the correct relative transform between two set of points.
    Args:
        scene: 3xN numpy array of points
        model: 3xM numpy array of points
        max_iterations: max amount of iterations the algorithm can perform.
        tolerance: tolerance before the algorithm converges.
    Returns:
      X_BA: A RigidTransform object that maps point_cloud_A on to point_cloud_B
            such that
                        X_BA.multiply(model) ~= scene,
      mean_error: Mean of all pairwise distances.
      num_iters: Number of iterations it took the ICP to converge.
    """
    X_BA = RigidTransform()

    mean_error = 0
    num_iters = 0
    prev_error = 0

    while True:
        num_iters += 1

        # your code here
        ##################
        distances, indices = nearest_neighbors(scene, model)
        model_correspondences = model[:, indices]
        
        X_BA_iter = least_squares_transform(scene, model_correspondences) # current iter transform
        
        model = X_BA_iter.multiply(model) # update model with 1 step transform
        X_BA = X_BA_iter.multiply(X_BA) # update total transform with 1 step transform

        mean_error = np.mean(distances)
        print(mean_error)
        ##################

        if abs(mean_error - prev_error) < tolerance or num_iters >= max_iterations:
            print("BREAK")
            break

        prev_error = mean_error

        meshcat.SetTransform("red_bunny", X_BA)
        print(f"iter {num_iters}")

    return X_BA, mean_error, num_iters

Now you should be able to visualize the registration of the Stanford bunny! Have fun!

In [107]:
icp(pointcloud_scene, pointcloud_model, max_iterations=30, tolerance=1e-5)

[3943 3943 4113 ... 5757 3943 3943]
[[-0.0295643 -0.0295643 -0.0282948 ... -0.0618399 -0.0295643 -0.0295643]
 [-0.0107509 -0.0107509 -0.0114934 ... -0.062019  -0.0107509 -0.0107509]
 [ 0.129853   0.129853   0.132551  ...  0.12397    0.129853   0.129853 ]]
0.09571969427875732
iter 1
[6819 7100 1604 ...  206  630 5989]
[[-0.08883143 -0.08538928 -0.06819638 ... -0.11196761 -0.10289977
  -0.09041815]
 [ 0.03736462  0.03533643 -0.0118319  ... -0.01559019 -0.01335856
   0.03837267]
 [ 0.15417758  0.15178624  0.13552863 ...  0.14653676  0.14144828
   0.15992352]]
0.02791382375828196
iter 2
[6591 2413  318 ... 3381 4038 5579]
[[-0.0874798  -0.0837319  -0.06955094 ... -0.12071121 -0.09923049
  -0.09737622]
 [ 0.03420942  0.03025978 -0.00390108 ... -0.0088826  -0.00365356
   0.03987568]
 [ 0.15341239  0.15003438  0.14030403 ...  0.14979923  0.1433431
   0.16929754]]
0.021226001509613825
iter 3
[2320  974 4127 ... 5757 3999 1066]
[[-0.08867112 -0.08719223 -0.07043738 ... -0.12621661 -0.0982296
  

(RigidTransform(
   R=RotationMatrix([
     [1.0000000000000002, -3.0861343802734856e-16, 1.5380437735721174e-16],
     [-3.07225943870068e-16, 0.866025403784436, -0.4999999999999992],
     [7.242470512203212e-17, 0.5, 0.8660254037844378],
   ]),
   p=[-0.1, 0.09999999999999992, 0.10000000000000007],
 ),
 1.7964638633936421e-16,
 27)

## How will this notebook be Graded?

If you are enrolled in the class, this notebook will be graded using [Gradescope](www.gradescope.com). You should have gotten the enrollement code on our announcement in Piazza.

For submission of this assignment, you must:
- Download and submit the notebook `bunny_icp.ipynb` to Gradescope's notebook submission section, along with your notebook for the other problems.

We will evaluate the local functions in the notebook to see if the function behaves as we have expected. For this exercise, the rubric is as follows:
- [3 pts] `least_squares_transform` must be implemented correctly.
- [3 pts] `icp` must be implemented correctly.

In [108]:
Grader.grade_output([TestICP], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 6/6.

Score for Test icp implementation is 3/3.
- [6978  323 4554 ... 2027 2115 1282]
[[-0.0192984   0.00135951 -0.00915874 ...  0.0425038   0.0378677
   0.0407274 ]
 [-0.0367502  -0.027808   -0.0684223  ... -0.0249931  -0.0367043
  -0.0325015 ]
 [ 0.0275938   0.074679    0.065113   ...  0.054374    0.057819
   0.0225207 ]]
0.023702749348665623
iter 1
[1584 2116 4025 ... 1967 2023 1361]
[[ 0.03046938  0.02741745 -0.01100214 ...  0.0309727   0.0300739
   0.03110757]
 [-0.02150489 -0.02693545 -0.07281363 ... -0.02438431 -0.03260405
  -0.03565129]
 [ 0.03284439  0.05590617  0.0611384  ...  0.05024555  0.05153303
   0.02396785]]
0.017752201957490426
iter 2
[1583 2070 4218 ... 2066 2064 1434]
[[ 0.02452407  0.02295977 -0.00948247 ...  0.02613479  0.02581234
   0.02836334]
 [-0.01692571 -0.02119126 -0.07183621 ... -0.02980327 -0.03592108
  -0.03709123]
 [ 0.03257643  0.05353076  0.05927264 ...  0.05242751  0.05281257
   0.02587388]]
0.014502491025865006
iter 3
[1716 2069  877 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cc6340f5-374e-449a-a195-839a3cedec4a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>